# Import a Dataset Into Jupyter

## importing libraries…

In [1]:
# Importing libraries
import pandas as pd
import numpy as np

import time, sys
from IPython.display import clear_output
from pathlib import Path 
import os

import matplotlib

from statsmodels.tsa.seasonal import seasonal_decompose

# plotly packages
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.tools as tls
from plotly.subplots import make_subplots

## Specify URL

url: https://data.cityofchicago.org/resource/5neh-572f.json

## Choosing a station

Print out all stations

In [2]:
station_id_set=pd.read_json('https://data.cityofchicago.org/resource/5neh-572f.json?$where=date%20between%20%272018-03-05T00:00:00%27%20and%20%272018-03-07T00:00:00%27&$select=distinct%20station_id&$limit=50000')

In [3]:
unique_station_id=[]
# using iteritems() function to retrieve rows
for value in station_id_set.itertuples():
    unique_station_id.append(value[1])
unique_station_id.sort()
print(unique_station_id)
print(len(unique_station_id))

[40010, 40020, 40030, 40040, 40050, 40060, 40070, 40080, 40090, 40100, 40120, 40130, 40140, 40150, 40160, 40170, 40180, 40190, 40200, 40210, 40220, 40230, 40240, 40250, 40260, 40270, 40280, 40290, 40300, 40310, 40320, 40330, 40340, 40350, 40360, 40370, 40380, 40390, 40400, 40420, 40430, 40440, 40450, 40460, 40470, 40480, 40490, 40510, 40520, 40530, 40540, 40550, 40560, 40570, 40580, 40590, 40600, 40610, 40630, 40650, 40660, 40670, 40680, 40690, 40700, 40710, 40720, 40730, 40740, 40750, 40760, 40770, 40780, 40790, 40800, 40810, 40820, 40830, 40840, 40850, 40870, 40880, 40890, 40900, 40910, 40920, 40930, 40940, 40960, 40970, 40980, 40990, 41000, 41010, 41020, 41030, 41040, 41050, 41060, 41070, 41080, 41090, 41120, 41130, 41140, 41150, 41160, 41170, 41180, 41190, 41200, 41210, 41220, 41230, 41240, 41250, 41260, 41270, 41280, 41290, 41300, 41310, 41320, 41330, 41340, 41350, 41360, 41380, 41400, 41410, 41420, 41430, 41440, 41450, 41460, 41480, 41490, 41500, 41510, 41660, 41670, 41680, 41690

## Page through data

| Query Parameter | Description | Default Value | Maximum Value |
| --- | --- | --- | --- |
| $limit | The number of results to return | 1000 | 50,000 |
| $offset | The index of the result array where to start the returned list of results. | 0 | N/A |

The order of the results of a query are not implicitly ordered, so if you're paging, make sure you provide an $order clause or at a minimum $order=:id. That will guarantee that the order of your results will be stable as you page through the dataset.

eg:
1) get the first 5 results: 'https://soda.demo.socrata.com/resource/earthquakes.json?$limit=5&$offset=0&$order=earthquake_id'

2) retrieve the next five results: 'https://soda.demo.socrata.com/resource/earthquakes.json?$limit=5&$offset=5&$order=earthquake_id'

In [4]:
def page_through_ridership_data(station_id,start_time,end_time):

    '''
    Name of Function: page_through_data
    Purpose of Function: to read the traffic data page by page
    Inputs: 
            -station_id
            -start_time
            -end_time
    Expected Outputs: 
            -station_id_xxxx_df
    ''' 

    # The $offset is the number of records into a dataset that you want to start, indexed at 0.
    offset  = 0 # start with no offset
    # segment_id = i #choose a road segment
    df_list = []
    data = pd.read_json(f"https://data.cityofchicago.org/resource/5neh-572f.json?$where=date%20between%20%27{start_time}%27%20and%20%27{end_time}%27&station_id={station_id}&$limit=50000&$offset={offset}&$order=date")
    #Notice the speed!=-1 condition specified in the url, it can help get rid of empty data
    while len(data)==50000: #check num of rows in each page, if num of rows == maximum value for $limit, next page exist
        df_list.append(data) #append data to the list
        offset = offset + 50000 #move to next page
        data = pd.read_json(f"https://data.cityofchicago.org/resource/sxs8-h27x.json?$where=date%20between%20%27{start_time}%27%20and%20%27{end_time}%27&station_id={station_id}&$limit=50000&$offset={offset}&$order=date")
    df_list.append(data)
    globals()['station_id_'+str(station_id) + '_df'] = pd.concat(df_list) #merge data into one dataset

In [5]:
station_id = 40890
page_through_ridership_data(station_id,'2018-03-05T00:00:00','2021-03-05T00:00:00')

In [6]:
globals()['station_id_'+str(station_id) + '_df'].head()

,station_id,stationname,date,daytype,rides
0,40890,O'Hare Airport,2018-03-05,W,9879
1,40890,O'Hare Airport,2018-03-06,W,9429
2,40890,O'Hare Airport,2018-03-07,W,9193
3,40890,O'Hare Airport,2018-03-08,W,10088
4,40890,O'Hare Airport,2018-03-09,W,10942


In [7]:
globals()['station_id_'+str(station_id) + '_df'].shape

(1097, 5)

In [13]:
fig = go.Figure(data=go.Scatter(x=globals()['station_id_'+str(station_id) + '_df'].date , y=globals()['station_id_'+str(station_id) + '_df'].rides, mode='lines'), layout = go.Layout(title=str(globals()['station_id_'+str(station_id) + '_df'].iloc[0]['stationname'])+' ridership', xaxis_title="date",yaxis_title="rides"))
fig.show()

## Time Series Decomposition 

Decompose time series into trend, seasonal, and residual components
- Trend — general movement over time
- Seasonal — behaviors captured in individual seasonal periods
- Residual — everything not captured by trend and seasonal components

The additive model is Y[t] = T[t] + S[t] + r[t]

The results are obtained by first estimating the trend by applying a convolution filter to the data. The trend is then removed from the series and the average of this de-trended series for each period is the returned seasonal component.

To get rid of the predictable ridership change, we only use the residual part

In [ ]:
def ridership_decompose(segment_id):
    """
    A function that returns the trend, seasonality and residual captured by applying additive model.
    Inputs:
            -station_id
    Expected Outputs:
            -station_id_xxxx_decompose
    """
    globals()['station_id_'+str(station_id) + '_df'].index = pd.to_datetime(globals()['station_id_'+str(station_id) + '_df'].date) # transform the time column into an actual date object and set it as index
    globals()['station_id_'+str(station_id) + '_decompose'] = seasonal_decompose(globals()['station_id_'+str(station_id) + '_df']['rides'], model = 'additive')

ridership_decompose(station_id)

In [ ]:
globals()['station_id_'+str(station_id) + '_decompose'].resid.head(20)

In [ ]:
fig = make_subplots(rows=4, cols=1,subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"])
fig.add_trace(go.Scatter(x=globals()['station_id_'+str(station_id) + '_decompose'].observed.index, y=globals()['station_id_'+str(station_id) + '_decompose'].observed, mode='lines'),row=1, col=1,)
fig.add_trace(go.Scatter(x=globals()['station_id_'+str(station_id) + '_decompose'].trend.index, y=globals()['station_id_'+str(station_id) + '_decompose'].trend, mode='lines'),row=2, col=1,)
fig.add_trace(go.Scatter(x=globals()['station_id_'+str(station_id) + '_decompose'].seasonal.index, y=globals()['station_id_'+str(station_id) + '_decompose'].seasonal, mode='lines'),row=3, col=1,)
fig.add_trace(go.Scatter(x=globals()['station_id_'+str(station_id) + '_decompose'].resid.index, y=globals()['station_id_'+str(station_id) + '_decompose'].resid, mode='lines'),row=4, col=1,)
fig.update_layout(height=900, margin=dict(t=100), showlegend=False)
fig.update_xaxes(showticklabels=False) # hide all the xticks
fig.update_xaxes(showticklabels=True, row=4, col=1)
fig.show()

## Calculate Ridership Reduction Index (RRI)

RRI = (1-Rac/Rff)*10

where consider the 85th percentile of the ridership as the free-flow ridership

In [ ]:
def RRI_Cal(station_id):
    
    '''
    Name of Function: RRI_Cal
    Purpose of Function: RRI calculation
        Inputs:
            -station_id
    Expected Outputs:
            -station_id_xxxx_resample
    '''    
    
    minimum = globals()['station_id_'+str(station_id) + '_decompose'].resid.min()
    resid = globals()['station_id_'+str(station_id) + '_decompose'].resid.subtract(minimum) #offset resid to all positive number
    quatile = resid.quantile(q=0.85) #calculate 85th percentile of the speed
    globals()['station_id_'+str(station_id) + '_df']['RRI']= (1-resid/quatile)*10
    # Notice here the RRI is stored in station_id_xxx_df


RRI_Cal(station_id)

In [ ]:
globals()['station_id_'+str(station_id) + '_df'].head(20)

Notice the NaN in the RRI column for the very begining and end of the time serie

In [ ]:
fig = go.Figure(data=go.Scatter(x=globals()['station_id_'+str(station_id) + '_df'].index , y=globals()['station_id_'+str(station_id) + '_df'].RRI, mode='lines'), layout = go.Layout(xaxis_title="time",yaxis_title="RRI"))
fig.show()

## Calculate RRI for all station

In [ ]:
# use a function to show progress
def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

# This cell may take 8 hr to finish
# Can change the for loop to only calculate few stations
i = 0
for station_id in unique_station_id:
    if os.path.exists(Path('RRI/station_' + str(station_id) + '_RRI.csv')): #check if SRI fila already exist in the SRI folder
        i = i+1
        update_progress(i / len(unique_station_id))
    else:
        page_through_ridership_data(station_id,'2018-03-05T00:00:00','2021-03-05T00:00:00')
        if len(globals()['station_id_'+str(station_id) + '_df'])==0:
            i=i+1
            update_progress(i / len(unique_station_id))
        else:
            ridership_decompose(station_id)
            RRI_Cal(station_id)
            filepath = Path('RRI/station_' + str(station_id) + '_RRI.csv')
            globals()['station_id_'+str(station_id) + '_df']['RRI'].to_csv(filepath)
            lst = [globals()['station_id_'+str(station_id) + '_df'], globals()['station_id_'+str(station_id) + '_decompose']]
            del globals()['station_id_'+str(station_id) + '_df'], globals()['station_id_'+str(station_id) + '_decompose'] # delete dataframe
            del lst # memory release
            i=i+1
            update_progress(i / len(unique_station_id))